Basics of Deep Learning assignment - Train Notebook

---
Written by:

- Matan Ofri
- Itamar Kirsch

In [ ]:
import keras as ker
import numpy as np
import os
from google.colab import drive
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import EarlyStopping , ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, Flatten, BatchNormalization, Dropout, Dense, UpSampling2D, Conv2DTranspose
from keras import initializers,layers, Input, backend, Model
from keras.initializers import glorot_normal
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import plot_model



Load Data from Google Drive


Please enter the path to the Basic folder

In [ ]:
PATH='/content/drive/Shareddrives/DeepLearning/Basics/'

🛑**Please note - you must load this block:**

In [ ]:
drive.mount('/content/drive')
#Section A - Binary Classification
val_binary = PATH+'Kaggle Data/chest_xray_2/val'
train_binary = PATH+'Kaggle Data/chest_xray_2/train'
#Section B - Multi-Class
val_multiclass =PATH+'Kaggle Data/chest_xray_3/val'
train_multiclass =PATH+'Kaggle Data/chest_xray_3/train'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load data for local use on RTX4090 (GPU) with Docker:

In [ ]:
#Section A - Binary Classification
val_binary = '/chest_xray/val'
train_binary = '/chest_xray/train'
#Section B - Multi-Class
val_multiclass ='/chest_xray3/val'
train_multiclass ='/chest_xray3/train'
if os.path.exists(val_binary)& os.path.exists(val_multiclass):
  print("Val data is loaded")
else:
  print("There is a problem loading validation data")
if os.path.exists(train_binary) & os.path.exists(train_multiclass):
  print("Train data is loaded")
else:
  print("There is a problem loading train data")

In [ ]:
#Section A - Binary Classification
val_binary = '/root/chest_xray_2/val'
train_binary = '/root/chest_xray_2/train'
#Section B - Multi-Class
val_multiclass ='/root/chest_xray_3/val'
train_multiclass ='/root/chest_xray_3/train'
if os.path.exists(val_binary)& os.path.exists(val_multiclass):
  print("Val data is loaded")
else:
  print("There is a problem loading validation data")
if os.path.exists(train_binary) & os.path.exists(train_multiclass):
  print("Train data is loaded")
else:
  print("There is a problem loading train data")

# Question number 1, Section A - Binary Classification

---

In [ ]:
batch_size = 20
img_width = 256
img_height = 256
epochs = 50

1. Data Augmentation (Preprocessing Data)

In [ ]:
binary_data_datagen = ImageDataGenerator(
   rescale=1./255,
   rotation_range=20,
   width_shift_range=0.2,
   height_shift_range=0.2,
   shear_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True,
   vertical_flip=True,
   fill_mode='constant',
   cval = 0,
)

binary_train_generator = binary_data_datagen.flow_from_directory(
    train_binary,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 26,
    #The samples are arranged in folders according to label, so we shuffle them:
    shuffle = True,
    class_mode='binary',
    color_mode='grayscale'
)

binary_validation_generator = binary_data_datagen.flow_from_directory(
    val_binary,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 26,
    shuffle = True,
    class_mode='binary',
    color_mode='grayscale'
)

2. Architecture - Binary Classification

In [ ]:
model = ker.Sequential()
#32 Inputs Block:
model.add(Conv2D(32, 3, activation='relu', strides=2, padding = 'same', name='Conv2D_input', input_shape=(img_height, img_width, 1),kernel_initializer=glorot_normal()))
model.add(MaxPooling2D(2, name='MaxPooling2D_input'))
#64-A Inputs Block:
model.add(Conv2D(64, 3, activation='relu', strides=2, padding = 'same', name='Conv2D_1'))
model.add(MaxPooling2D(2, name='MaxPooling2D_1'))
#128 Inputs Block:
model.add(Conv2D(128, 3, activation='relu', strides=2, padding = 'same', name='Conv2D_2'))
model.add(MaxPooling2D(2, name='MaxPooling2D_2'))
#64-B Inputs Block:
model.add(Conv2D(64, 3, activation='relu', strides=2, padding = 'same', name='Conv2D_3'))
model.add(MaxPooling2D(2, name='MaxPooling2D_3'))

model.add(Flatten(name='Flatten'))
model.add(Dense(64, activation='relu', name='Dense_1'))
#Binary classification, so we use 1 unit and 'sigmoid' activation:
model.add(Dense(1, activation='sigmoid', name='Dense_output'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

3. The model training function

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3,  restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',mode='auto', factor=0.5, patience=2, verbose=1)

history = model.fit(
    binary_train_generator,
    steps_per_epoch = binary_train_generator.samples//batch_size,
    epochs = epochs,
    validation_data = binary_validation_generator,
    validation_steps = binary_validation_generator.samples//batch_size,
    callbacks=[early_stopping,reduce_lr],
    verbose=2
)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_n = range(1, len(train_loss) + 1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(epochs_n, train_loss, 'bo-', label='Training loss')
plt.plot(epochs_n, val_loss, 'ro-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Question number 1, Section B - Multi-Class Classification

---





In [ ]:
batch_size = 32
img_width = 96
img_height = 96
epochs = 50

1. Data Augmentation (Preprocessing Data)

In [ ]:
multiclass_data_datagen = ImageDataGenerator(
     rescale=1./255,
     rotation_range=30,
     width_shift_range=0.1,
     height_shift_range=0.1,
     zoom_range=0.2,
     horizontal_flip=False,
     vertical_flip=False,
)

multiclass_train_generator = multiclass_data_datagen.flow_from_directory(
    train_multiclass,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    #Change for multiclass:
    class_mode='categorical',
)

validation_generator = multiclass_data_datagen.flow_from_directory(
    val_multiclass,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    #Change for multiclass:
    class_mode='categorical',
)

2. Creating initial weights according to the amount of samples in every class

In [ ]:
print(multiclass_train_generator.class_indices)
temp = Counter(multiclass_train_generator.classes)
sum = temp[0]+ temp[1]+ temp[2]
BACTERIA_WEIGHT=((1/temp[0])*(sum/3.0))
NORMAL_WEIGHT=((1/temp[1])*(sum/3.0))
VIRUS_WEIGHT=((1/temp[2])*(sum/3.0))
ALL_WEIGHT={0: BACTERIA_WEIGHT,1:NORMAL_WEIGHT,2:VIRUS_WEIGHT}

3. Architecture - Multi-Class Classification

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', name='Conv2D_input',input_shape = (img_height, img_width, 3)))
model.add(MaxPooling2D((2,2), name='MaxPooling2D_input'))

model.add(Conv2D(64, (3,3), activation = 'relu', name='Conv2D_1'))
model.add(MaxPooling2D((2,2), name='MaxPooling2D_1'))
model.add(Dropout(0.2, name='Dropout_1'))

model.add(Conv2D(128, (3,3), activation = 'relu', name='Conv2D_2'))
model.add(MaxPooling2D((2,2), name='MaxPooling2D_2'))
model.add(Dropout(0.2, name='Dropout_2'))

model.add(Conv2D(256 , (3,3), activation = 'relu', name='Conv2D_3'))
model.add(MaxPooling2D((2,2), name='MaxPooling2D_3'))

model.add(Flatten(name='Flaten'))
model.add(Dropout(0.2, name='Dropuot_flatten'))
model.add(Dense(256 , activation = 'relu', name='Dense_flatten'))
model.add(Dense(3, activation='softmax', name='Dense_output'))
model.compile(optimizer = RMSprop(learning_rate=0.001, rho=0.90, epsilon=1e-08),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

4. The model training function

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=7,  restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',mode='auto', factor=0.5, patience=2, verbose=1)

history = model.fit(
    multiclass_train_generator,
    steps_per_epoch = multiclass_train_generator.samples//batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//batch_size,
    class_weight=ALL_WEIGHT,
    callbacks=[early_stopping,reduce_lr],
    verbose=2
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_n = range(1, len(train_loss) + 1)

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs_n = range(2, len(train_loss) + 1)
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
train_loss.pop(0)
val_loss.pop(0)
plt.plot(epochs_n, train_loss, 'bo-', label='Training loss')
plt.plot(epochs_n, val_loss, 'ro-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

5. Save Embedding Vector

In [ ]:
#Save Embedding Vector from H5 file:
from keras.models import model_from_json
model_json = embedding_model.to_json()
with open("embedding_model.json", "w") as json_file:
  json_file.write(model_json)
embedding_model.save_weights("embedding_multicalss_vector.h5")

# Question number 2 - KNN with Embedding Vector

---





In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from keras.models import load_model
from keras.models import model_from_json
from joblib import dump ,load
import random

1. Load our Classification Model and Create the Embedding Model

🛑**Please note - you must load this block:**

In [ ]:
model = load_model(PATH+'Models/KNN.keras')
embedding_model = Sequential()
for layer in model.layers[:-1]:
 embedding_model.add(layer)
embedding_model.compile()

2. Load Data

In [ ]:
img_height = 300
img_width = 300
batch_size=20

datagen = ImageDataGenerator(rescale=1./255)

validation_set_knn = datagen.flow_from_directory(
    val_multiclass,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle = False
)
train_set_knn = datagen.flow_from_directory(
    train_multiclass,
    target_size = (img_height, img_width),
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False
)

- Note - Creating the embedded vectors on the Google Colab takes over 30 minutes
- For testing purposes we will load the embedded predictions in the next block

In [ ]:
  #embeddings1 = embedding_model.predict(train_set_knn)
  #embeddings2 = embedding_model.predict(validation_set_knn)
  #allembeddings=[]
  #allembeddings = np.concatenate([embeddings1, embeddings2], axis=0)
  #alllabels = np.concatenate([train_set_knn.classes, validation_set_knn.classes], axis=0)

🛑**Please note - you must load this block:**

In [ ]:
allembeddings= np.load(PATH+'Embeddings/allembeddings.npy')
alllabels= np.load(PATH+'Embeddings/alllabels.npy')

3. Construction of the KNN model


In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=72,weights='distance',metric='hamming')
knn_classifier.fit(allembeddings, alllabels)

4. Save KNN Model for test notebook

In [ ]:
knnFile = open(PATH+'Models/knnpickle_file', 'wb')
pickle.dump(knn_classifier, knnFile)
knnFile.close()

5. Create a 3D graph with t-SNE library

In [ ]:
class_names = {
    0: 'Bacteria',
    1: 'Normal',
    2: 'Virus'
}

# Reduce dimensionality using t-SNE
tsne = TSNE(n_components=3)
allembeddings_tsne = tsne.fit_transform(allembeddings)

# scaling factors for each component
scale_factors = [2.0, 1.5, 1.0]

# Scale t-SNE components
allembeddings_tsne_scaled = allembeddings_tsne * scale_factors

# Plot the data points in 3D
fig = plt.figure(figsize=(30, 30))

ax = fig.add_subplot(111, projection='3d')
for label in np.unique(alllabels):
    ax.scatter(allembeddings_tsne_scaled[alllabels == label, 0],
               allembeddings_tsne_scaled[alllabels == label, 1],
               allembeddings_tsne_scaled[alllabels == label, 2],
               label=class_names[label], s=20, alpha=0.5)

ax.view_init(elev=30, azim=45)
ax.set_title('t-SNE Visualization of X-ray Scans in 3D: Dividing Images into 3 Classes')
ax.legend(fontsize=20)
plt.show()

In [ ]:
fig = plt.figure(figsize=(30, 30))
ax = fig.add_subplot(111, projection='3d')
for label in np.unique(alllabels):
    ax.scatter(allembeddings_tsne_scaled[alllabels == label, 0],
               allembeddings_tsne_scaled[alllabels == label, 1],
               allembeddings_tsne_scaled[alllabels == label, 2],
               label=class_names[label], s=20, alpha=0.5)

ax.view_init(elev=30, azim=45)
ax.set_title('t-SNE Visualization of X-ray Scans in 3D: Dividing Images into 3 Classes')
ax.legend(fontsize=20)
plt.show()

# Question number 3 - Identifying anomalies in X-ray images

---

1. Load data for anomaly detection model

🛑**Please note - you must load this block:**

In [ ]:
drive.mount('/content/drive')
train_dir = PATH+'Kaggle Data/anomaly/train'
val_dir = PATH+'Kaggle Data/anomaly/val'
anom_dir = PATH+'Kaggle Data/anomaly/anom'
if os.path.exists(train_dir)&os.path.exists(anom_dir)& os.path.exists(val_dir):
  print("Data is loaded")
else:
  print("There is a problem loading data")

In [ ]:
batch_size = 20
img_height = 384
img_width = 384
latent_dim = 40

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input',

)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input',


)

anomaly_generator = train_datagen.flow_from_directory(
   anom_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input',

)

2. Encoder Model

In [ ]:
# There is no need to run this section - you can load a ready-made model below
input_shape = (img_height, img_width, 1)
inputs = Input(shape=input_shape,name='Input')

x = inputs
x = Conv2D(8, 1, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D1')(x)
x = Conv2D(16, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D2')(x)
x = MaxPooling2D(2, padding = 'same',name='MaxPooling1')(x)
x = Conv2D(32, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D3')(x)
x = MaxPooling2D(2, padding = 'same',name='MaxPooling2')(x)
x = Conv2D(64, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D4')(x)
x = MaxPooling2D(2, padding = 'same',name='MaxPooling3')(x)
x = Conv2D(128, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D5')(x)
x = MaxPooling2D(2, padding = 'same',name='MaxPooling4')(x)
x = Conv2D(256, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D6')(x)
x = MaxPooling2D(2, padding = 'same',name='MaxPooling5')(x)
x = Conv2D(512, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2D7')(x)
x = MaxPooling2D(2, padding = 'same',name='MaxPooling6')(x)
shape = backend.int_shape(x)
x = Flatten(name='Flatten')(x)
latent = Dense(latent_dim,name='output')(x)
encoder = Model(inputs,latent)
encoder.summary()

3. Decoder Model

In [ ]:
# There is no need to run this section - you can load a ready-made model below
latent_inputs = Input(shape=(latent_dim,),name='input')
x = Dense(shape[1]*shape[2]*shape[3],name="Dense")(latent_inputs)
x = ker.layers.Reshape((shape[1],shape[2],shape[3]),name='Reshape')(x)
x = UpSampling2D((2,2),name='Upsampling2D1')(x)
x = Conv2DTranspose(512, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2DTranspose1')(x)
x = UpSampling2D((2,2),name='Upsampling2D2')(x)
x = Conv2DTranspose(256, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2DTranspose2')(x)
x = UpSampling2D((2,2),name='Upsampling2D3')(x)
x = Conv2DTranspose(128, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2DTranspose3')(x)
x = UpSampling2D((2,2),name='Upsampling2D4')(x)
x = Conv2DTranspose(64, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2DTranspose4')(x)
x = UpSampling2D((2,2),name='Upsampling2D5')(x)
x = Conv2DTranspose(32, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2DTranspose5')(x)
x = UpSampling2D((2,2),name='Upsampling2D6')(x)
x = Conv2DTranspose(16, 3, activation=LeakyReLU(alpha=0.3), padding = 'same',name='Conv2DTranspose6')(x)
x = Conv2DTranspose(8, 1, activation=backend.tanh, padding = 'same',name='Conv2DTranspose7')(x)
output = Conv2DTranspose(1, (1,1), activation='sigmoid', padding = 'same',name='Output')(x)
decoder = Model(latent_inputs, output)
decoder.summary()

In [ ]:
plot_model(decoder, to_file='encoder.png', show_shapes=True, show_layer_names=True)

4. Autoencoder model

In [ ]:
# There is no need to run this section - you can load a ready-made model below
autoencoder = Model(inputs,decoder(encoder(inputs)))
autoencoder.compile(optimizer=Adam(learning_rate=0.0005, beta_1=0.5, beta_2=0.9), loss='mse', metrics='accuracy',run_eagerly=True)#We should try SSIM loss
autoencoder.summary()

5. Loading a pre-trained model

🛑**Please note - you must load this block:**

In [ ]:
autoencoder=ker.models.load_model(PATH+'Models/autoencoder.keras')
autoencoder.summary()

6. Viewing X-ray images of lungs alongside images reproduced by the decoder


In [ ]:
data_list = []
batch_index = 0
while batch_index <= train_generator.batch_index:
    data = train_generator.next()
    data_list.append(data[0])
    batch_index = batch_index + 1
predicted = autoencoder.predict(train_generator)
no_of_samples = 4
_, axs = plt.subplots(no_of_samples, 3, figsize=(5, 8))
axs = axs.flatten()
imgs = []
for i in range(no_of_samples):
    imgs.append(data_list[0][i])
    imgs.append(predicted[i])
    imgs.append(np.abs(predicted[i]-data_list[0][i]))
for img, ax in zip(imgs, axs):
    ax.imshow(img,cmap='magma')
plt.show()

7. Creating embeddings vectors


In [ ]:
# There is no need to run this section - you can load a ready-made vectors below
prediction_loss2=[]
prediction_loss1=[]
prediction_loss3=[]




image_files = [os.path.join(train_dir+'/NORMAL', file) for file in os.listdir(train_dir+'/NORMAL') if file.endswith('.jpeg')]

for dirs in image_files:
  img = image.load_img(dirs, target_size=(img_height, img_width))
  gray_img = img.convert('L')
  img_array = image.img_to_array(gray_img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = img_array / 255.0
  reconstructed_image = autoencoder.predict(img_array)
  prediction_loss1.append(np.mean(np.abs((reconstructed_image -img_array))))


image_files = [os.path.join(val_dir+'/NORMAL', file) for file in os.listdir('/content/anomaly/Normal_val/NORMAL') if file.endswith('.jpeg')]

for dirs in image_files:
  img = image.load_img(dirs, target_size=(img_height, img_width))
  gray_img = img.convert('L')
  img_array = image.img_to_array(gray_img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = img_array / 255.0
  reconstructed_image = autoencoder.predict(img_array)
  prediction_loss3.append(np.mean(np.abs((reconstructed_image -img_array))))


selected_class = 'Sick'
image_files = [os.path.join(anom_dir+'/Sick', file) for file in os.listdir(anom_dir+'/Sick') if file.endswith('.jpeg')]

for dirs in image_files:
  img = image.load_img(dirs, target_size=(img_height, img_width))
  gray_img = img.convert('L')
  img_array = image.img_to_array(gray_img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = img_array / 255.0
  reconstructed_image = autoencoder.predict(img_array)

  prediction_loss2.append(np.mean(np.abs((reconstructed_image -img_array))))

# Flatten the ndarray to create a 1D list of values and append the flattened values to a larger list:
test_reconstruction_values = []
anom_reconstruction_values = []
val_reconstruction_values = []

for ndarray in prediction_loss1:
    flattened_values = ndarray.flatten()
    test_reconstruction_values.extend(flattened_values)

for ndarray in prediction_loss2:
    flattened_values = ndarray.flatten()
    anom_reconstruction_values.extend(flattened_values)
for ndarray in prediction_loss3:
    flattened_values = ndarray.flatten()
    val_reconstruction_values.extend(flattened_values)



🛑**Please note - you must load this block:**

In [ ]:
train_and_val_flattened_values= np.load(PATH+'Embeddings/train_and_val_flattened_values.npy')
anomaly_flattened_values= np.load(PATH+'Embeddings/anomaly_flattened_values.npy')

8. A graph showing the loss relative to the amount of samples for which the value was obtained


In [ ]:
loss_threshold_test=  np.percentile(test_reconstruction_values, 98)
print(f'The prediction loss threshold train for 2% of the outliers is {loss_threshold_test}')
loss_threshold_anom= np.percentile(anom_reconstruction_values, 98)
print(f'The prediction loss threshold anom for 2% of the outliers is {loss_threshold_anom}')

loss_threshold_val= np.percentile(val_reconstruction_values, 98)
print(f'The prediction loss threshold val for 2% of the outliers is {loss_threshold_val}')
flat1=np.array(test_reconstruction_values).reshape(-1,1)
flat2=np.array(anom_reconstruction_values).reshape(-1,1)
flat3=np.array(val_reconstruction_values).reshape(-1,1)
plt.hist(flat1,alpha=0.5,color='green')
plt.hist(flat2,alpha=0.5,color='red')
plt.hist(flat3,alpha=0.5,color='yellow')
plt.axvline(x=loss_threshold_test,color= 'green')
plt.axvline(x=loss_threshold_anom,color= 'red')
plt.axvline(x=loss_threshold_val,color= 'yellow')
plt.xlabel('Prediction Loss')
plt.ylabel('Frequency')